In [1]:
import os
import random
import numpy as np

def set_seed(seed: int):
    random.seed(seed) # Python
    np.random.seed(seed)  # Numpy, é o gerador utilizado pelo sklearn
    os.environ["PYTHONHASHSEED"] = str(seed)  # sistema operativo

set_seed(25)

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # No max width

In [8]:
df = pd.read_csv("data/csvs/flow_data_gerador_cpp.csv")

In [5]:
df = df.drop_duplicates()

In [6]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,34978,0,1,0,292,0,292,292,292.0,0.0,0,9223372036854775807,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0,9223372036854775807,0,0.000000,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,1,0,0,0,278,0,0.000000,0.000000,292,292,292.0,0.000000,0.000000,0,0,0,1,1,0,0,0,0.0,292.0,292.0,0.0,0,0,0,0,0,0,1,292,0,0,501,0,1,292,0,0,0,0,0,0,0,0
1,34958,0,1,0,278,0,278,278,278.0,0.0,0,9223372036854775807,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0,9223372036854775807,0,0.000000,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,1,0,0,0,264,0,0.000000,0.000000,278,278,278.0,0.000000,0.000000,0,0,0,1,1,0,0,0,0.0,278.0,278.0,0.0,0,0,0,0,0,0,1,278,0,0,501,0,1,278,0,0,0,0,0,0,0,0
2,34962,0,1,0,275,0,275,275,275.0,0.0,0,9223372036854775807,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0,9223372036854775807,0,0.000000,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,1,0,0,0,261,0,0.000000,0.000000,275,275,275.0,0.000000,0.000000,0,0,0,1,1,0,0,0,0.0,275.0,275.0,0.0,0,0,0,0,0,0,1,275,0,0,501,0,1,275,0,0,0,0,0,0,0,0
3,34978,4,2,1,567,78,292,275,283.5,8.5,78,78,78.0,0.0,1.292383e+02,0.601108,2.495390,2.495382,4,0,4,4.990781,0.0,4,4,0,0.0,0.0,0,9223372036854775807,2,0,0,0,539,64,0.400739,0.200369,78,292,215.0,6.940221,48.166667,0,0,0,2,3,0,0,0,0.5,215.0,283.5,78.0,0,0,0,0,0,0,2,567,1,78,501,625,2,275,0,0,0,0,0,0,0,0
4,34978,0,1,1,292,78,292,292,292.0,0.0,78,78,78.0,0.0,4.702704e+07,254200.242424,0.000008,0.000000,0,0,0,0.000000,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,1,0,0,0,278,64,127100.121212,127100.121212,78,292,185.0,0.000000,0.000000,0,0,0,1,2,0,0,0,1.0,185.0,292.0,78.0,0,0,0,0,0,0,1,292,1,78,501,625,1,292,0,0,0,0,0,0,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129977 entries, 0 to 129976
Data columns (total 77 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Destination Port             129977 non-null  int64  
 1   Flow Duration                129977 non-null  int64  
 2   Total Fwd Packets            129977 non-null  int64  
 3   Total Backward Packets       129977 non-null  int64  
 4   Total Length of Fwd Packets  129977 non-null  int64  
 5   Total Length of Bwd Packets  129977 non-null  int64  
 6   Fwd Packet Length Max        129977 non-null  int64  
 7   Fwd Packet Length Min        129977 non-null  int64  
 8   Fwd Packet Length Mean       129977 non-null  float64
 9   Fwd Packet Length Std        129977 non-null  float64
 10  Bwd Packet Length Max        129977 non-null  int64  
 11  Bwd Packet Length Min        129977 non-null  int64  
 12  Bwd Packet Length Mean       129977 non-null  float64
 13 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/csvs/flow_data_gerador_cpp.csv")
X_train, X_test = train_test_split(df, test_size=0.2, random_state=25)

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('anomaly_detector', IsolationForest(n_estimators=300, max_features=1.0, contamination=0.01,random_state=25))
])

pipeline.fit(X_train)

# Predict on test data
y_pred_labels = pipeline.predict(X_test)

y_pred_labels = np.where(y_pred_labels == 1,"BENIGN","ANOMALY")

unique, counts = np.unique(y_pred_labels, return_counts=True)

print(dict(zip(unique, counts)))

{np.str_('ANOMALY'): np.int64(4446), np.str_('BENIGN'): np.int64(21550)}


In [14]:
import joblib

# Save the best model to a file
joblib.dump(pipeline, 'isolation_model_trafego_gerado.pkl')

['isolation_model_trafego_gerado.pkl']